In [28]:
pip install gensim numpy pandas nltk textblob scipy sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip

--2022-06-29 13:42:24--  https://dl.fbaipublicfiles.com/fasttext/vectors-english/wiki-news-300d-1M.vec.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 172.67.9.4, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 681808098 (650M) [application/zip]
Saving to: ‘wiki-news-300d-1M.vec.zip’

wiki-news-300d-1M.v 100%[===================>] 650.22M  22.7MB/s    in 31s     

2022-06-29 13:42:56 (20.9 MB/s) - ‘wiki-news-300d-1M.vec.zip’ saved [681808098/681808098]



In [8]:
!unzip "/content/wiki-news-300d-1M.vec.zip"

Archive:  /content/wiki-news-300d-1M.vec.zip
  inflating: wiki-news-300d-1M.vec   


In [9]:
model = gensim.models.KeyedVectors.load_word2vec_format('/content/wiki-news-300d-1M.vec')

In [10]:
index2word_set = set(model.wv.index2word)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


In [34]:
from nltk import WordPunctTokenizer
from textblob.base import BaseTokenizer
import numpy as np
from scipy import spatial
from sklearn.metrics import pairwise_distances
import pickle


class NLTKWordPunctTokenizer(BaseTokenizer):
    def tokenize(self, text):
        return WordPunctTokenizer().tokenize(text)


def avg_feature_vector(sentence, num_features=300):
    words = NLTKWordPunctTokenizer().tokenize(sentence)
    feature_vec = np.zeros((num_features,), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec


def cosine_similarity(vec1, vec2):
    return 1 - spatial.distance.cosine(vec1, vec2)


def sent2sent_similarity(sentence1, sentence2):
    vec1, vec2 = avg_feature_vector(sentence1), avg_feature_vector(sentence2)
    return cosine_similarity(vec1, vec2)


In [35]:
sentences = ["No message..no responce..what happend?",
             "At WHAT TIME should i come tomorrow",
             "Come to my home for one last time i wont do anything. Trust me.",
             "See you there!",
             "Great. So should i send you my account number.",
             "Do we have any spare power supplies",
             "Ok try to do week end course in coimbatore.",
             "I have lost 10 kilos as of today!",
             "Still chance there. If you search hard you will get it..let have a try :)",
             "Night night, see you tomorrow",
             "Get ur 1st RINGTONE FREE NOW! Reply to this msg with TONE. Gr8 TOP 20 tones to your phone every week just £1.50 per wk 2 opt out send STOP 08452810071 16",
             "URGENT! You have won a 1 week FREE membership in our £100,000 Prize Jackpot! Txt the word: CLAIM to No: 81010 T&C www.dbuk.net LCCLTD POBOX 4403LDNW1A7RW18",
             "YES! The only place in town to meet exciting adult singles is now in the UK. Txt CHAT to 86688 now! 150p/Msg.",
             "URGENT!: Your Mobile No. was awarded a £2,000 Bonus Caller Prize on 02/09/03! This is our 2nd attempt to contact YOU! Call 0871-872-9755 BOX95QU",
             "Ever thought about living a good life with a perfect partner? Just txt back NAME and AGE to join the mobile community. (100p/SMS)",
             "We tried to contact you re your reply to our offer of a Video Phone 750 anytime any network mins Half Price Line Rental Camcorder Reply or call 08000930705",
             "Free Top ringtone -sub to weekly ringtone-get 1st week free-send SUBPOLY to 81618-?3 per week-stop sms-08718727870",
             "Your weekly Cool-Mob tones are ready to download !This weeks new Tones include: 1) Crazy Frog-AXEL F>>> 2) Akon-Lonely>>> 3) Black Eyed-Dont P >>>More info in n",
             "Bought one ringtone and now getting texts costing 3 pound offering more tones etc",
             "URGENT We are trying to contact you Last weekends draw shows u have won a £1000 prize GUARANTEED Call 09064017295 Claim code K52 Valid 12hrs 150p pm"]
labels = ["ham"] * 10 + ["spam"] * 10

sentence_embeddings = list(map(lambda x: avg_feature_vector(x), sentences))

with open('sentence_embeddings.pickle', 'wb') as handle:
    pickle.dump(sentence_embeddings, handle, protocol=pickle.HIGHEST_PROTOCOL)

pairwise_cosine_dists = pairwise_distances(sentence_embeddings, metric=cosine_similarity)
pairwise_cosine_dists.shape
pairwise_cosine_dists

with open('pairwise_cosine_dists.pickle', 'wb') as handle:
    pickle.dump(pairwise_cosine_dists, handle, protocol=pickle.HIGHEST_PROTOCOL)